In [2]:
from pathlib import Path
import numpy as np
path_to_data = Path("/media/hdd1/i-care/10s")

In [5]:
short_seq = []
for patient in list(path_to_data.iterdir()):
    for eeg_file in patient.iterdir():
        eeg = np.load(eeg_file)
        if eeg.shape[0] < 1000:
            short_seq.append(eeg_file.name)